In [26]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle 

In [27]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [28]:
# Preprocess the data 
data = data.drop(columns=['RowNumber','CustomerId','Surname'], axis=1)

In [29]:
# Encode the categorical variables 
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])


In [30]:
# One Hot Encoding Geography 
one_hot_encoder = OneHotEncoder()
geo_encoded = one_hot_encoder.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [31]:
#Combine the one-hot encoded columns with the original dataframe
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [32]:
# Split the data into features and target 
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [33]:
# Train Test Split 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Scale this feature 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [34]:
# save the encoders and scalers for later use 
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder,file)


with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)




ANN Regression Problem Statement

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [36]:
# Build The Model
model = Sequential([
    Dense(64, activation='relu',input_shape = (X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1) # Output layer for regression 
])


# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()

c:\Users\87504\Desktop\MachineLearning\annclassification\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime 

# Set up TensorBoard

log_dir = "regression_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [38]:
# Set Up Early Stopping 
early_stopping_Callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [39]:
# train the model 
history = model.fit(
    X_train,y_train,
    validation_data = (X_test,y_test),
    epochs=100,
    callbacks = [early_stopping_Callback,tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 13381164032.0000 - mae: 100365.3594 - val_loss: 12998622208.0000 - val_mae: 98476.0781
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 13208961024.0000 - mae: 99509.5156 - val_loss: 12660913152.0000 - val_mae: 96772.5703
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 12618009600.0000 - mae: 96575.0703 - val_loss: 11810758656.0000 - val_mae: 92472.3672
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 11462079488.0000 - mae: 90859.2266 - val_loss: 10407684096.0000 - val_mae: 85413.4766
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9803591680.0000 - mae: 82560.6172 - val_loss: 8616977408.0000 - val_mae: 76443.6250
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 7899575296.0000 - mae: 73041.5234 - val_loss: 6766234112.0000 - val_mae: 67161.2422
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6119555584.0000 - mae: 64107.9492 - val_loss: 5231285248.000

In [40]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [43]:
%tensorboard --logdir regression_logs/fit

Reusing TensorBoard on port 6006 (pid 24692), started 0:01:09 ago. (Use '!kill 24692' to kill it.)

In [44]:
## Evaluate the model on test data 
test_loss,test_mae = model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3354929920.0000 - mae: 50111.0273
Test MAE : 50111.02734375
